<a href="https://colab.research.google.com/github/tuhinmallick/AI-for-Fashion/blob/main/Training%2C_Loading%2C_and_Merging_QDoRA%2C_QLoRA%2C_and_LoftQ_Adapters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is organized into two sections.

The first section shows how to fine-tune QLoRA, QDoRA, and LoftQ adapters for Mistral 7B using Hugging Face PEFT and TRL libraries. Then, the notebook benchmarks the adapters before and after merging them into the base model, followed by quantization with BNB NF4 and AWQ.


First, we need all these dependencies:

In [ ]:
!pip install -q -U transformers peft accelerate datasets trl bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.1 MB/s eta 0:00:00


Import all the necessary packages.

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig
)
from trl import SFTTrainer

# Fine-tuning

**LoftQ**

More info:
[LoftQ: Better Initialization for a Quantization-Aware LoRA](https://kaitchup.substack.com/p/loftq-better-initialization-for-a)


In [ ]:
!git clone https://github.com/huggingface/peft.git

!python peft/examples/loftq_finetuning/quantize_save_load.py \
    --model_name_or_path mistralai/Mistral-7B-v0.1 \
    --bits 4 \
    --iter 5 \
    --rank 16 \
    --save_dir "./loftq_iters/"

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  (v_proj): lora.Linear(
    (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
    (lora_dropout): ModuleDict(
      (default): Dropout(p=0.05, inplace=False)
    )
    (lora_A): ModuleDict(
      (default): Linear(in_features=4096, out_features=16, bias=False)
    )
    (lora_B): ModuleDict(
      (default): Linear(in_features=16, out_features=1024, bias=False)
    )
    (lora_embedding_A): ParameterDict()
    (lora_embedding_B): ParameterDict()
  )
  (o_proj): Shell()
  (rotary_emb): MistralRotaryEmbedding()
)
MistralMLP(
  (gate_proj): lora.Linear(
    (base_layer): Linear(in_features=4096, out_features=14336, bias=False)
    (lora_dropout): ModuleDict(
      (default): Dropout(p=0.05, inplace=False)
    )
    (lora_A): ModuleDict(
      (default): Linear(in_features=4096, out_features=16, bias=False)
    )
    (lora_B): ModuleDict(
      (default): Linear(in_features=16, out_features=1

In [ ]:
MODEL_DIR = "./loftq_iters/Mistral-7B-v0.1-4bit-16rank"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, add_eos_token=True, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
          MODEL_DIR, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=torch.bfloat16
)

model.save_pretrained("./loftq_base/")
tokenizer.save_pretrained("./loftq_base/")


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

('./drive/MyDrive/mergexp/loftq_base/tokenizer_config.json',
 './drive/MyDrive/mergexp/loftq_base/special_tokens_map.json',
 './drive/MyDrive/mergexp/loftq_base/tokenizer.model',
 './drive/MyDrive/mergexp/loftq_base/added_tokens.json',
 './drive/MyDrive/mergexp/loftq_base/tokenizer.json')

In [ ]:
!mkdir -p ./drive/MyDrive/mergexp/loftq_base/

In [ ]:
!cp -r ./loftq_iters/Mistral-7B-v0.1-4bit-16rank/loft_init/ ./drive/MyDrive/mergexp/loftq_base/loft_init/

In [ ]:
MODEL_DIR = "./loftq_base/"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' #Necessary for FlashAttention compatibility

#Better to use bf16 if supported (Ampere GPUs or more recent)
#If bf16 is supported, the GPU is also recent enough to support FlashAttention
if torch.cuda.is_bf16_supported():
  !pip install flash_attn
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

dataset = load_dataset("timdettmers/openassistant-guanaco")

model = AutoModelForCausalLM.from_pretrained(
          MODEL_DIR,  device_map={"": 0}, torch_dtype=compute_dtype
)
model = prepare_model_for_kbit_training(model)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id

peft_model = PeftModel.from_pretrained(
    model,
    MODEL_DIR,
    subfolder="loft_init",
    is_trainable=True,
)



from trl import SFTConfig

training_arguments = SFTConfig(
        output_dir="./loftq_ft/",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        log_level="debug",
        logging_steps=50,
        learning_rate=1e-5,
        eval_steps=50,
        num_train_epochs=1,
        fp16= not torch.cuda.is_bf16_supported(),
        bf16= torch.cuda.is_bf16_supported(),
        save_strategy='epoch',
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
)

trainer = SFTTrainer(
        model=peft_model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 18.8 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.5.6-cp310-cp310-linux_x86_64.whl size=120592258 sha256=d8cf54adda65f59820221d329d274e124972d7fdc05ab3b1130253c64eee6c8a
  Stored in directory: /root/.cache/pip/wheels/a8/1c/88/b959d6818b98a46d61ba231683abb7523b89ac1a7ed1e0c206
Successfully built flash_attn


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


...

Step,Training Loss,Validation Loss
50,1.347700,1.354418
100,1.270200,1.237236
150,1.185200,1.192417
200,1.160600,1.178373
250,1.139600,1.162789
300,1.154300,1.151754
350,1.130600,1.148634
400,1.138800,1.145539
450,1.123100,1.142953
500,1.120400,1.141005


***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differe

Step,Training Loss,Validation Loss
50,1.347700,1.354418
100,1.270200,1.237236
150,1.185200,1.192417
200,1.160600,1.178373
250,1.139600,1.162789
300,1.154300,1.151754
350,1.130600,1.148634
400,1.138800,1.145539
450,1.123100,1.142953
500,1.120400,1.141005


***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differe

TrainOutput(global_step=1231, training_loss=1.1346433749535914, metrics={'train_runtime': 15766.5706, 'train_samples_per_second': 0.624, 'train_steps_per_second': 0.078, 'total_flos': 2.148541273402245e+17, 'train_loss': 1.1346433749535914, 'epoch': 1.0})

Merge the LoftQ adapter

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
#Better to use bf16 if supported (Ampere GPUs or more recent)
#If bf16 is supported, the GPU is also recent enough to support FlashAttention
if torch.cuda.is_bf16_supported():
  !pip install flash_attn
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'


model_name = "./loftq_base/"
adapter = "./loftq_ft/checkpoint-1231"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_size="left",  use_fast=True)

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)

model = dequantize_model(model, to='./dqz_model_loftq/',dtype=compute_dtype)
model = PeftModel.from_pretrained(model, adapter)
model = model.merge_and_unload()

print(f"Successfully loaded the model {model_name} into memory")

#pushed to hub
#model.push_to_hub("kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged")
#tokenizer.push_to_hub("kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged")


/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Dequantizing `model.layers.0.self_attn.q_proj`...
Dequantizing `model.layers.0.self_attn.k_proj`...
Dequantizing `model.layers.0.self_attn.v_proj`...
Dequantizing `model.layers.0.self_attn.o_proj`...
Dequantizing `model.layers.0.mlp.gate_proj`...
Dequantizing `model.layers.0.mlp.up_proj`...
Dequantizing `model.layers.0.mlp.down_proj`...
Dequantizing `model.layers.1.self_attn.q_proj`...
Dequantizing `model.layers.1.self_attn.k_proj`...
Dequantizing `model.layers.1.self_attn.v_proj`...
Dequantizing `model.layers.1.self_attn.o_proj`...
Dequantizing `model.layers.1.mlp.gate_proj`...
Dequantizing `model.layers.1.mlp.up_proj`...
Dequantizing `model.layers.1.mlp.down_proj`...
Dequantizing `model.layers.2.self_attn.q_proj`...
Dequantizing `model.layers.2.self_attn.k_proj`...
Dequantizing `model.layers.2.self_attn.v_proj`...
Dequantizing `model.layers.2.self_attn.o_proj`...
Dequantizing `model.layers.2.mlp.gate_proj`...
Dequantizing `model.layers.2.mlp.up_proj`...
Dequantizing `model.layers.2.m

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged/commit/fb41de77231ac8904d1f195cd8b28cec80a22add', commit_message='Upload tokenizer', commit_description='', oid='fb41de77231ac8904d1f195cd8b28cec80a22add', pr_url=None, pr_revision=None, pr_num=None)



---

**QDoRA**

More info:
[DoRA: Better and Faster than LoRA?](https://kaitchup.substack.com/p/dora-better-and-faster-than-lora)

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' #Necessary for FlashAttention compatibility

#Better to use bf16 if supported (Ampere GPUs or more recent)
#If bf16 is supported, the GPU is also recent enough to support FlashAttention
if torch.cuda.is_bf16_supported():
  !pip install flash_attn
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

dataset = load_dataset("timdettmers/openassistant-guanaco")


#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' #Necessary for FlashAttention compatibility

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
model = prepare_model_for_kbit_training(model)
model.gradient_checkpointing_enable({'use_reentrant':True})
#model.enable_input_require_grads()

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        use_dora=True,
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)


from trl import SFTConfig

training_arguments = SFTConfig(
        output_dir="./qdora/",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        log_level="debug",
        logging_steps=50,
        learning_rate=1e-5,
        eval_steps=50,
        num_train_epochs=1,
        fp16= not torch.cuda.is_bf16_supported(),
        bf16= torch.cuda.is_bf16_supported(),
        save_strategy='epoch',
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
)

trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        dataset_text_field="text",
        max_seq_length=512,
        peft_config=peft_config,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/tokenizer_config.json


...

Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Using auto half precision backend
Currently training with a batch size of: 8
***** Running training *****
  Num examples = 9,846


Step,Training Loss,Validation Loss
50,1.347300,1.353635
100,1.262400,1.231624
150,1.180400,1.188604
200,1.155300,1.169926
250,1.128800,1.153853
300,1.150000,1.150144
350,1.127900,1.147451
400,1.138100,1.145200
450,1.122600,1.143323
500,1.121200,1.141987


***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  N

Merging the adapter

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
#Better to use bf16 if supported (Ampere GPUs or more recent)
#If bf16 is supported, the GPU is also recent enough to support FlashAttention
if torch.cuda.is_bf16_supported():
  !pip install flash_attn
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'


model_name = "mistralai/Mistral-7B-v0.1"
adapter = "./qdora/checkpoint-1231"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_size="left",  use_fast=True)

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)

model = dequantize_model(model, to='./dqz_model_qdora/',dtype=compute_dtype)
model = PeftModel.from_pretrained(model, adapter)
model = model.merge_and_unload()

print(f"Successfully loaded the model {model_name} into memory")
#pushed to the hub
#model.push_to_hub("kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-qdora-merged")
#tokenizer.push_to_hub("kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-qdora-merged")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Dequantizing `model.layers.0.self_attn.q_proj`...
Dequantizing `model.layers.0.self_attn.k_proj`...
Dequantizing `model.layers.0.self_attn.v_proj`...
Dequantizing `model.layers.0.self_attn.o_proj`...
Dequantizing `model.layers.0.mlp.gate_proj`...
Dequantizing `model.layers.0.mlp.up_proj`...
Dequantizing `model.layers.0.mlp.down_proj`...
Dequantizing `model.layers.1.self_attn.q_proj`...
Dequantizing `model.layers.1.self_attn.k_proj`...
Dequantizing `model.layers.1.self_attn.v_proj`...
Dequantizing `model.layers.1.self_attn.o_proj`...
Dequantizing `model.layers.1.mlp.gate_proj`...
Dequantizing `model.layers.1.mlp.up_proj`...
Dequantizing `model.layers.1.mlp.down_proj`...
Dequantizing `model.layers.2.self_attn.q_proj`...
Dequantizing `model.layers.2.self_attn.k_proj`...
Dequantizing `model.layers.2.self_attn.v_proj`...
Dequantizing `model.layers.2.self_attn.o_proj`...
Dequantizing `model.layers.2.mlp.gate_proj`...
Dequantizing `model.layers.2.mlp.up_proj`...
Dequantizing `model.layers.2.m

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-qdora-merged/commit/558bbd6ffcc2df825d10398546ed0566511e9653', commit_message='Upload tokenizer', commit_description='', oid='558bbd6ffcc2df825d10398546ed0566511e9653', pr_url=None, pr_revision=None, pr_num=None)



---


**QLoRA**

More info: [QLoRA: Fine-Tune a Large Language Model on Your GPU](https://kaitchup.substack.com/p/qlora-fine-tune-a-large-language-model-on-your-gpu-27bed5a03e2b)

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' #Necessary for FlashAttention compatibility

#Better to use bf16 if supported (Ampere GPUs or more recent)
#If bf16 is supported, the GPU is also recent enough to support FlashAttention
if torch.cuda.is_bf16_supported():
  !pip install flash_attn
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

dataset = load_dataset("timdettmers/openassistant-guanaco")


#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' #Necessary for FlashAttention compatibility

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
model = prepare_model_for_kbit_training(model)
model.gradient_checkpointing_enable({'use_reentrant':True})
#model.enable_input_require_grads()

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)


from trl import SFTConfig

training_arguments = SFTConfig(
        output_dir="./qlora/",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        log_level="debug",
        logging_steps=50,
        learning_rate=1e-5,
        eval_steps=50,
        num_train_epochs=1,
        fp16= not torch.cuda.is_bf16_supported(),
        bf16= torch.cuda.is_bf16_supported(),
        save_strategy='epoch',
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
)

trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        dataset_text_field="text",
        max_seq_length=512,
        peft_config=peft_config,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.0 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.5.7-cp310-cp310-linux_x86_64.whl size=120853563 sha256=bbe6f77fd0899f8a125a5bdcf734b660c4c88e81c9b51c7ce98ebeba44dc6fa0
  Stored in directory: /root/.cache/pip/wheels/13/96/ed/bcac89c56b606421f99b45b16a94db5d0f2b6b4eaf8bac4d01
Successfully built flash_attn


...

Step,Training Loss,Validation Loss
50,1.347300,1.353649
100,1.262000,1.231738
150,1.181200,1.189125
200,1.156100,1.170896
250,1.129900,1.154133
300,1.150300,1.150258
350,1.128000,1.147490
400,1.138300,1.145134
450,1.122700,1.143200
500,1.121100,1.141903


***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8


Step,Training Loss,Validation Loss
50,1.347300,1.353649
100,1.262000,1.231738
150,1.181200,1.189125
200,1.156100,1.170896
250,1.129900,1.154133
300,1.150300,1.150258
350,1.128000,1.147490
400,1.138300,1.145134
450,1.122700,1.143200
500,1.121100,1.141903


***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
Saving model checkpoint to ./drive/MyDrive/mergexp/qlora/checkpoint-1231
loading configuration file con

TrainOutput(global_step=1231, training_loss=1.1352182507611792, metrics={'train_runtime': 15574.6019, 'train_samples_per_second': 0.632, 'train_steps_per_second': 0.079, 'total_flos': 2.148541273402245e+17, 'train_loss': 1.1352182507611792, 'epoch': 1.0})

Merging the adapter

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
#Better to use bf16 if supported (Ampere GPUs or more recent)
#If bf16 is supported, the GPU is also recent enough to support FlashAttention
if torch.cuda.is_bf16_supported():
  !pip install flash_attn
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'


model_name = "mistralai/Mistral-7B-v0.1"
adapter = "./qlora/checkpoint-1231"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_size="left",  use_fast=True)

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)

model = dequantize_model(model, to='./dqz_model_qlora/',dtype=compute_dtype)
model = PeftModel.from_pretrained(model, adapter)
model = model.merge_and_unload()

print(f"Successfully loaded the model {model_name} into memory")

#pushed to the hub
#model.push_to_hub("kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged")
#tokenizer.push_to_hub("kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Dequantizing `model.layers.0.self_attn.q_proj`...
Dequantizing `model.layers.0.self_attn.k_proj`...
Dequantizing `model.layers.0.self_attn.v_proj`...
Dequantizing `model.layers.0.self_attn.o_proj`...
Dequantizing `model.layers.0.mlp.gate_proj`...
Dequantizing `model.layers.0.mlp.up_proj`...
Dequantizing `model.layers.0.mlp.down_proj`...
Dequantizing `model.layers.1.self_attn.q_proj`...
Dequantizing `model.layers.1.self_attn.k_proj`...
Dequantizing `model.layers.1.self_attn.v_proj`...
Dequantizing `model.layers.1.self_attn.o_proj`...
Dequantizing `model.layers.1.mlp.gate_proj`...
Dequantizing `model.layers.1.mlp.up_proj`...
Dequantizing `model.layers.1.mlp.down_proj`...
Dequantizing `model.layers.2.self_attn.q_proj`...
Dequantizing `model.layers.2.self_attn.k_proj`...
Dequantizing `model.layers.2.self_attn.v_proj`...
Dequantizing `model.layers.2.self_attn.o_proj`...
Dequantizing `model.layers.2.mlp.gate_proj`...
Dequantizing `model.layers.2.mlp.up_proj`...
Dequantizing `model.layers.2.m

# Benchmarking: Inference throughtput and accuracy

Some utilities to dequantize a model and benchmark perplexity and inference speed.

The dequantization is done before merging the adapter. More info here:
[LoRA Adapters: When a Naive Merge Leads to Poor Performance](https://kaitchup.substack.com/p/lora-adapters-when-a-naive-merge)

In [ ]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from datasets import load_dataset
from peft.utils import _get_submodules
import os,json, copy
import bitsandbytes as bnb
from bitsandbytes.functional import dequantize_4bit
import time
#Better to use bf16 if supported (Ampere GPUs or more recent)
#If bf16 is supported, the GPU is also recent enough to support FlashAttention
if torch.cuda.is_bf16_supported():
  !pip install flash_attn
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'


def dequantize_model(model, to='./dequantized_model', dtype=torch.float16, device="cuda"):
    """
    'model': the peftmodel you loaded with qlora.
    'tokenizer': the model's corresponding hf's tokenizer.
    'to': directory to save the dequantized model
    'dtype': dtype that the model was trained using
    'device': device to load the model to
    """


    os.makedirs(to, exist_ok=True)

    cls = bnb.nn.Linear4bit

    with torch.no_grad():
        for name, module in model.named_modules():
            if isinstance(module, cls):
                print(f"Dequantizing `{name}`...")
                quant_state = copy.deepcopy(module.weight.quant_state)
                quant_state.dtype = dtype

                weights = dequantize_4bit(module.weight.data, quant_state=quant_state, quant_type="nf4").to(dtype)

                new_module = torch.nn.Linear(module.in_features, module.out_features, bias=None, dtype=dtype)
                new_module.weight = torch.nn.Parameter(weights)
                new_module.to(device=device, dtype=dtype)

                parent, target, target_name = _get_submodules(model, name)
                setattr(parent, target_name, new_module)

        # a hack, setting this to avoid hf's saving error because hf
        # itself does not support saving a model that is registered to be loaded in 4bit.
        model.is_loaded_in_4bit = False

        print("Saving dequantized model...")
        model.save_pretrained(to)
        #tokenizer.save_pretrained(to)
        config_data = json.loads(open(os.path.join(to, 'config.json'), 'r').read())
        config_data.pop("quantization_config", None)
        config_data.pop("pretraining_tp", None)
        with open(os.path.join(to, 'config.json'), 'w') as config:
            config.write(json.dumps(config_data, indent=2))

        return model


#compute the perplexity on a dataset given a tokenizer and a model
def ppl_model(model, tokenizer, dataset):
  nlls= []
  max_length = 2048
  stride = 512
  for s in tqdm(range(len(dataset['text']))):
      encodings = tokenizer(dataset['text'][s], return_tensors="pt")
      seq_len = encodings.input_ids.size(1)
      prev_end_loc = 0
      for begin_loc in range(0, seq_len, stride):
          end_loc = min(begin_loc + max_length, seq_len)
          trg_len = end_loc - prev_end_loc
          input_ids = encodings.input_ids[:, begin_loc:end_loc].to("cuda")
          target_ids = input_ids.clone()
          target_ids[:, :-trg_len] = -100
          with torch.no_grad():
              outputs = model(input_ids, labels=target_ids)
              neg_log_likelihood = outputs.loss
          nlls.append(neg_log_likelihood)
          prev_end_loc = end_loc
          if end_loc == seq_len:
              break
  ppl = torch.exp(torch.stack(nlls).mean())
  return ppl.item()

model_name = "mistralai/Mistral-7B-v0.1"

#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_size="left",  use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
p = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. \n\n Tell me about gravity."


def eval_model(model):
  total_tokens = 0
  total_duration = 0
  for b in range(5):


    inputs = tokenizer(p, return_tensors="pt").to("cuda")
    generation_time = time.time()
    outputs = model.generate(**inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=300)
    duration = time.time() - generation_time
    total_duration += duration

    for output in outputs:
      result = tokenizer.decode(output)
      nb_tokens = len(result)
      total_tokens += nb_tokens
    print("--- Speed: %s tokens/second ---" % (round(nb_tokens/duration,2)))
  print("--- Average speed: %s tokens/second ---" % (round(total_tokens/total_duration,2)))

  ds = load_dataset("timdettmers/openassistant-guanaco", split='test')
  ppl = ppl_model(model, tokenizer, ds)
  print("Perplexity: "+str(ppl))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 19.2 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.5.7-cp310-cp310-linux_x86_64.whl size=120853563 sha256=bbe6f77fd0899f8a125a5bdcf734b660c4c88e81c9b51c7ce98ebeba44dc6fa0
  Stored in directory: /root/.cache/pip/wheels/13/96/ed/bcac89c56b606421f99b45b16a94db5d0f2b6b4eaf8bac4d01
Successfully built flash_attn


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

# Baseline: Mistral 7B Quantized with bitsandbytes NF4

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
loading_start = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
eval_model(model)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

100%|██████████| 518/518 [02:18<00:00,  3.73it/s]

Perplexity: 4.720216751098633


# QLoRA

With a loaded adapter

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
adapter = "../qlora/checkpoint-1231"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
loading_start = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
loading_adapter_start = time.time()
model = PeftModel.from_pretrained(model, adapter)
print("--- Loading adapter time: %s seconds ---" % (time.time() - loading_adapter_start))

eval_model(model)


model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

--- Loading model time: 64.71461462974548 seconds ---
--- Loading adapter time: 6.66745662689209 seconds ---
--- Speed: 43.31 tokens/second ---
--- Speed: 45.56 tokens/second ---
--- Speed: 45.41 tokens/second ---
--- Speed: 45.01 tokens/second ---
--- Speed: 45.22 tokens/second ---
--- Average speed: 44.89 tokens/second ---


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

100%|██████████| 518/518 [02:52<00:00,  3.01it/s]

Perplexity: 3.379533052444458


with a merged adapter

In [ ]:
model_name = "kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged"
loading_start = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_name, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
eval_model(model)


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

--- Loading model time: 7.132479667663574 seconds ---
--- Speed: 76.58 tokens/second ---
--- Speed: 76.71 tokens/second ---
--- Speed: 76.79 tokens/second ---
--- Speed: 76.68 tokens/second ---
--- Speed: 76.64 tokens/second ---
--- Average speed: 76.68 tokens/second ---


100%|██████████| 518/518 [01:36<00:00,  5.35it/s]

Perplexity: 3.3805997371673584


With merged adapter and BNB NF4 quantization

In [ ]:
model_name = "kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged"
loading_start = time.time()
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
eval_model(model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

--- Loading model time: 7.103466033935547 seconds ---
--- Speed: 34.68 tokens/second ---
--- Speed: 34.67 tokens/second ---
--- Speed: 34.91 tokens/second ---
--- Speed: 35.11 tokens/second ---
--- Speed: 35.0 tokens/second ---
--- Average speed: 34.87 tokens/second ---


100%|██████████| 518/518 [02:18<00:00,  3.74it/s]

Perplexity: 4.590646266937256


With merged adapter and AWQ quantization

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged'
quant_path = 'Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged-awq-4bit'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model and tokenizer
model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model with safetensors
model.save_quantized("./"+quant_path, safetensors=True)
tokenizer.save_pretrained("./"+quant_path)

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/993 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
AWQ: 100%|██████████| 32/32 [28:30<00:00, 53.45s/it]


('./Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged-awq-4bit/tokenizer_config.json',
 './Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged-awq-4bit/special_tokens_map.json',
 './Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged-awq-4bit/tokenizer.model',
 './Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged-awq-4bit/added_tokens.json',
 './Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged-awq-4bit/tokenizer.json')

In [ ]:
model_name = "Mistral-7B-v0.1-oasstguanaco-1e-qlora-merged-awq-4bit"
loading_start = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_name, device_map={"": 0}, torch_dtype=torch.float16
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
eval_model(model)

--- Loading model time: 2.4577279090881348 seconds ---
--- Speed: 278.74 tokens/second ---
--- Speed: 876.4 tokens/second ---
--- Speed: 113.87 tokens/second ---
--- Speed: 108.53 tokens/second ---
--- Speed: 435.08 tokens/second ---
--- Average speed: 152.05 tokens/second ---


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
100%|██████████| 518/518 [02:57<00:00,  2.92it/s]

Perplexity: 3.4783289432525635


# QDoRA
With a loaded adapter

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
adapter = "./qdora/checkpoint-1231"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
loading_start = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
loading_adapter_start = time.time()
model = PeftModel.from_pretrained(model, adapter)
print("--- Loading adapter time: %s seconds ---" % (time.time() - loading_adapter_start))

eval_model(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

--- Loading model time: 57.367106676101685 seconds ---
--- Loading adapter time: 6.7472243309021 seconds ---
--- Speed: 5.0 tokens/second ---
--- Speed: 5.01 tokens/second ---
--- Speed: 5.01 tokens/second ---
--- Speed: 5.01 tokens/second ---
--- Speed: 5.01 tokens/second ---
--- Average speed: 5.01 tokens/second ---


100%|██████████| 518/518 [18:27<00:00,  2.14s/it]

Perplexity: 3.3790879249572754


with a merged adapter

In [ ]:
model_name = "kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-qdora-merged"
loading_start = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_name, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
eval_model(model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

--- Loading model time: 5.826384544372559 seconds ---
--- Speed: 74.77 tokens/second ---
--- Speed: 77.26 tokens/second ---
--- Speed: 77.34 tokens/second ---
--- Speed: 77.63 tokens/second ---
--- Speed: 77.64 tokens/second ---
--- Average speed: 76.91 tokens/second ---


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
100%|██████████| 518/518 [01:37<00:00,  5.33it/s]

Perplexity: 3.3809475898742676


With merged adapter and BNB NF4 quantization

In [ ]:
model_name = "kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-qdora-merged"
loading_start = time.time()
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
eval_model(model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

--- Loading model time: 6.595882892608643 seconds ---
--- Speed: 35.01 tokens/second ---
--- Speed: 35.06 tokens/second ---
--- Speed: 35.07 tokens/second ---
--- Speed: 35.14 tokens/second ---
--- Speed: 35.1 tokens/second ---
--- Average speed: 35.08 tokens/second ---


100%|██████████| 518/518 [02:18<00:00,  3.74it/s]

Perplexity: 4.622631072998047


With merged adapter and AWQ quantization

In [ ]:
!pip install autoawq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 50.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3


In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-qdora-merged'
quant_path = 'Mistral-7B-v0.1-oasstguanaco-1e-qdora-merged-awq-4bit'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model and tokenizer
model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model with safetensors
model.save_quantized("./"+quant_path, safetensors=True)
tokenizer.save_pretrained("./"+quant_path)

In [ ]:
model_name = "Mistral-7B-v0.1-oasstguanaco-1e-qdora-merged-awq-4bit"
loading_start = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_name, device_map={"": 0}, torch_dtype=torch.float16
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
eval_model(model)

--- Loading model time: 2.804798126220703 seconds ---
--- Speed: 96.55 tokens/second ---
--- Speed: 118.2 tokens/second ---
--- Speed: 114.38 tokens/second ---
--- Speed: 1568.48 tokens/second ---
--- Speed: 6009.32 tokens/second ---
--- Average speed: 132.51 tokens/second ---


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

100%|██████████| 518/518 [02:57<00:00,  2.92it/s]

Perplexity: 3.477247714996338


# LoftQ

In [ ]:
model_name = "./loftq_base/"
adapter = "./loftq_ft/checkpoint-1231"

loading_start = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_name, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
loading_adapter_start = time.time()
model = PeftModel.from_pretrained(model, adapter)
print("--- Loading adapter time: %s seconds ---" % (time.time() - loading_adapter_start))

eval_model(model)

--- Loading model time: 51.703083515167236 seconds ---
--- Loading adapter time: 9.962267637252808 seconds ---
--- Speed: 40.67 tokens/second ---
--- Speed: 43.75 tokens/second ---
--- Speed: 43.7 tokens/second ---
--- Speed: 43.75 tokens/second ---
--- Speed: 43.64 tokens/second ---
--- Average speed: 43.07 tokens/second ---


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
100%|██████████| 518/518 [02:51<00:00,  3.01it/s]

Perplexity: 3.355402946472168


With merged adapter

In [ ]:
model_name = "kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged"
loading_start = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_name, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
eval_model(model)

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

--- Loading model time: 345.3683784008026 seconds ---
--- Speed: 73.65 tokens/second ---
--- Speed: 75.74 tokens/second ---
--- Speed: 75.76 tokens/second ---
--- Speed: 75.65 tokens/second ---
--- Speed: 75.77 tokens/second ---
--- Average speed: 75.3 tokens/second ---


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
100%|██████████| 518/518 [01:35<00:00,  5.45it/s]

Perplexity: 3.3551559448242188


With merged adapter and BNB NF4 quantization

In [ ]:
model_name = "kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged"
loading_start = time.time()
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, torch_dtype=compute_dtype,  attn_implementation=attn_implementation
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
eval_model(model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

--- Loading model time: 7.500432014465332 seconds ---
--- Speed: 30.09 tokens/second ---
--- Speed: 34.83 tokens/second ---
--- Speed: 35.02 tokens/second ---
--- Speed: 35.15 tokens/second ---
--- Speed: 35.07 tokens/second ---
--- Average speed: 33.91 tokens/second ---


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
100%|██████████| 518/518 [02:17<00:00,  3.76it/s]

Perplexity: 4.526394844055176


With merged adapter and GPTQ quantization

In [ ]:
!pip install --upgrade transformers auto-gptq accelerate datasets auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.0/410.0 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using c

In [ ]:
from transformers import  GPTQConfig

model_path = 'kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

quantization_config = GPTQConfig(bits=4, dataset = "c4", tokenizer=tokenizer)

model = AutoModelForCausalLM.from_pretrained(model_path, device_map={"": 0}, quantization_config=quantization_config)

tokenizer_config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [ ]:
eval_model(model)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


--- Speed: 65.54 tokens/second ---


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


--- Speed: 72.3 tokens/second ---


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

100%|██████████| 518/518 [00:35<00:00, 14.44it/s]

Perplexity: 3.671870470046997


With merged adapter and AWQ quantization

In [ ]:
!pip install autoawq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 50.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3


In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'kaitchup/Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged'
quant_path = 'Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged-awq-4bit'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model and tokenizer
model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model with safetensors
model.save_quantized("./"+quant_path, safetensors=True)
tokenizer.save_pretrained("./"+quant_path)

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating validation split: 0 examples [00:00, ? examples/s]

AWQ: 100%|██████████| 32/32 [28:31<00:00, 53.48s/it]


('./Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged-awq-4bit/tokenizer_config.json',
 './Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged-awq-4bit/special_tokens_map.json',
 './Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged-awq-4bit/tokenizer.model',
 './Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged-awq-4bit/added_tokens.json',
 './Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged-awq-4bit/tokenizer.json')

In [ ]:
model_name = "Mistral-7B-v0.1-oasstguanaco-1e-loftq-merged-awq-4bit"
loading_start = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_name, device_map={"": 0}, torch_dtype=torch.float16
)
print("--- Loading model time: %s seconds ---" % (time.time() - loading_start))
eval_model(model)

--- Loading model time: 2.324984312057495 seconds ---
--- Speed: 109.81 tokens/second ---
--- Speed: 156.0 tokens/second ---
--- Speed: 154.99 tokens/second ---
--- Speed: 158.94 tokens/second ---
--- Speed: 153.41 tokens/second ---
--- Average speed: 140.26 tokens/second ---


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

100%|██████████| 518/518 [02:58<00:00,  2.90it/s]

Perplexity: 3.4350991249084473
